In [1]:
import os
import sys
import json
import requests
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Add scripts folder to path so we can import canvas_api.py
sys.path.append('../scripts')

# Config values
CANVAS_DOMAIN = os.getenv("CANVAS_API_BASE")  # already ends in /api/v1
ACCESS_TOKEN = os.getenv("CANVAS_API_TOKEN")

# Example fallback (in case you're working in notebook outside the normal flow)
COURSE_ID = 1158826  # ART 001A sandbox; can override this later

CURRENT_TERM_ID = 202550  # Spring 2025 — update as needed

from IPython.display import display, HTML

display(HTML("""
<style>
.slo-button {
  border: 1px solid #ccc;
  border-radius: 6px;
  padding: 8px 14px;
  margin: 2px;
  font-size: 14px;
  cursor: pointer;
}

.slo-button.selected {
  border: 2px solid black;
}

.score-4 { background-color: #c6f6d5; }  /* green (Exceeded) */
.score-3 { background-color: #bee3f8; }  /* blue (Met) */
.score-2 { background-color: #fbd38d; }  /* orange (Partially Met) */
.score-1 { background-color: #feb2b2; }  /* red (Not Met) */
</style>
"""))



In [2]:
def get_slo_mapping(course_id):
    """
    Fetches slo_map.json from a course's Files area in Canvas.
    """
    headers = {"Authorization": f"Bearer {os.getenv('CANVAS_API_TOKEN')}"}
    canvas_domain = os.getenv("CANVAS_API_BASE").replace("/api/v1", "")
    
    # Step 1: Get list of files in the course
    list_files_url = f"{canvas_domain}/api/v1/courses/{course_id}/files"
    response = requests.get(list_files_url, headers=headers)
    response.raise_for_status()
    files = response.json()

    # Step 2: Find slo_map.json
    slo_file = next((f for f in files if f["filename"] == "slo_map.json"), None)
    if not slo_file:
        raise Exception("slo_map.json not found in course files.")
    
    # Step 3: Download the file
    download_url = slo_file["url"]
    file_response = requests.get(download_url, headers=headers)
    file_response.raise_for_status()
    
    return json.loads(file_response.text)

# Load the slo_map
slo_map = get_slo_mapping(COURSE_ID)
print("✅ Loaded slo_map.json")


✅ Loaded slo_map.json


In [3]:
def get_sections(course_id):
    """
    Returns a list of sections in a given course, including their SIS IDs.
    """
    headers = {"Authorization": f"Bearer {os.getenv('CANVAS_API_TOKEN')}"}
    canvas_domain = os.getenv("CANVAS_API_BASE").replace("/api/v1", "")
    
    url = f"{canvas_domain}/api/v1/courses/{course_id}/sections"
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    sections = response.json()

    return [
        {
            "id": section["id"],
            "sis_section_id": section.get("sis_section_id"),
            "name": section.get("name")
        }
        for section in sections
    ]

def parse_course_code_from_sis_id(sis_id):
    """
    Extracts course code from SIS ID like 'Sum25_ART001A_98765' → 'ART001A'
    """
    try:
        parts = sis_id.split("_")
        if len(parts) >= 3:
            return parts[1]
    except Exception as e:
        print(f"Error parsing SIS ID '{sis_id}': {e}")
    return "UNKNOWN"

# 🔍 Test it out
sections = get_sections(COURSE_ID)
for section in sections:
    parsed = parse_course_code_from_sis_id(section["sis_section_id"])
    print(f"SIS: {section['sis_section_id']} → Parsed Course Code: {parsed}")


SIS: Sum25_ART001A_98765 → Parsed Course Code: ART001A


In [4]:
# Place this near your API helper functions
def get_instructor_name():
    headers = {"Authorization": f"Bearer " + os.getenv("CANVAS_API_TOKEN")}
    url = os.getenv("CANVAS_API_BASE").replace("/api/v1", "") + "/api/v1/users/self/profile"
    
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    
    return response.json().get("name", "Unknown Instructor")


In [5]:
def is_user_teacher(course_id):
    """
    Returns True if the current user is enrolled in the course as a Teacher.
    """
    headers = {"Authorization": f"Bearer " + os.getenv("CANVAS_API_TOKEN")}
    canvas_domain = os.getenv("CANVAS_API_BASE").replace("/api/v1", "")
    
    url = f"{canvas_domain}/api/v1/courses/{course_id}/enrollments?user_id=self"
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    
    enrollments = response.json()
    for e in enrollments:
        if e.get("type") == "TeacherEnrollment":
            return True
    return False


In [6]:
def get_outcomes_for_course(slo_map, course_code, term_code=None):
    """
    Looks up outcomes for a given course_code and term_code in slo_map.
    Returns a dict with required and optional outcome IDs.
    """
    try:
        term_code = term_code or str(CURRENT_TERM_ID)  # fallback to global term ID
        course_entry = slo_map.get(course_code, {})
        term_entry = course_entry.get(term_code, {})
        return {
            "required": term_entry.get("required", []),
            "optional": term_entry.get("optional", [])
        }
    except Exception as e:
        print(f"Error retrieving outcomes for {course_code} / {term_code}: {e}")
        return {"required": [], "optional": []}

# ⛏ Extract outcomes based on parsed code
parsed_course_code = parse_course_code_from_sis_id(sections[0]["sis_section_id"])
outcomes = get_outcomes_for_course(slo_map, parsed_course_code)

print(f"✅ Outcomes for {parsed_course_code}:")
print("Required:", outcomes["required"])
print("Optional:", outcomes["optional"])


✅ Outcomes for ART001A:
Required: [1400300, 1400302]
Optional: [1400301, 1400303]


In [7]:
def get_students_in_section(section_id):
    """
    Fetches all students in a given section.
    """
    headers = {"Authorization": f"Bearer {os.getenv('CANVAS_API_TOKEN')}"}
    canvas_domain = os.getenv("CANVAS_API_BASE").replace("/api/v1", "")
    
    url = f"{canvas_domain}/api/v1/sections/{section_id}/enrollments?type[]=StudentEnrollment"
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    enrollments = response.json()

    return [
        {
            "id": e["user_id"],
            "name": e["user"]["name"],
            "sortable_name": e["user"].get("sortable_name"),
            "sis_user_id": e["user"].get("sis_user_id")
        }
        for e in enrollments if e["type"] == "StudentEnrollment"
    ]

# 👥 Choose a section (first one by default for now)
section_id = sections[0]["id"]

# 🎯 Get the students in that section
students = get_students_in_section(section_id)

print(f"✅ Students in section {section_id}:")
for s in students:
    print(f"- {s['name']} (ID: {s['id']})")


✅ Students in section 1303851:
- Pam Beesly (ID: 2216505)
- Jim Halpert (ID: 2216504)
- Angela Kinsey (ID: 2216503)
- Dwight Schrute (ID: 2216506)


In [8]:
import pandas as pd

# Load the outcomes CSV (assuming it lives in ../data)
outcomes_df = pd.read_csv("../data/slo_outcomes.csv")

# Normalize only outcome rows
def clean_guid(guid):
    if pd.isna(guid):
        return None
    guid = str(guid)
    if "canvas_outcome:" in guid:
        return guid.split(":")[-1].strip()
    return None

# Build the outcome_descriptions dictionary
outcome_descriptions = {
    cleaned: str(row["description"]).strip()
    for _, row in outcomes_df.iterrows()
    if (cleaned := clean_guid(row["vendor_guid"])) is not None
}

# Debug print
print("✅ Loaded outcome_descriptions:")
print("Sample keys:", list(outcome_descriptions.keys())[:5])

# Temporary hardcoded patch values (these may eventually come from CSV)
outcome_descriptions.update({
    "1400300": "Describe and analyze works of art and architecture, employing the language of formal and contextual analysis modeled in class and in assigned reading.",
    "1400301": "Identify the key stylistic and thematic features of a work of art or architecture from a given period, cultural tradition and regional location of prehistory through the middle ages, including unknown works.",
    "1400302": "Evaluate works of art and/or architecture, accounting for both similarities and differences.",
    "1400303": "Analyze the ways in which art from prehistory through the middle ages has been employed historically to express fundamental human ideals, values, and beliefs."
})


✅ Loaded outcome_descriptions:
Sample keys: ['1254565', '1255117', '1255118', '1255119', '1255120']


In [9]:
import ipywidgets as widgets
from IPython.display import display, HTML, FileLink
import re
import csv
import io

# === Reverse scoring options (4 to 1) ===
score_options = [
    ("Exceeded (4)", 4),
    ("Met (3)", 3),
    ("Partially Met (2)", 2),
    ("Not Met (1)", 1)
]

# === Extract CRN from section SIS ID ===
def extract_crn(sis_id):
    try:
        return sis_id.split("_")[-1]
    except:
        return "Unknown"

# === Extract formatted course name and SLO number from outcome title ===
def parse_course_and_slo_number(title):
    match = re.match(r"([A-Z]{3,4} \d+[A-Z]?) SLO (\d+)", title)
    if match:
        course = match.group(1)
        slo_number = match.group(2)
        return course, slo_number
    return "Unknown Course", "?"

# === Build the in-memory score UI ===
def render_slo_interface(course_code, section_id, students, outcomes, slo_metadata, outcome_descriptions, sections, instructor_name=None):
    student_scores = {}

    matching_section = next((s for s in sections if s["id"] == section_id), {})
    crn = extract_crn(matching_section.get("sis_section_id", "Unknown"))
    formatted_course_name = course_code if " " in course_code else re.sub(r"([A-Z]{3,4})(\d+)", r"\1 \2", course_code)

    display(HTML(f"<h3>SLO Scoring for {formatted_course_name} #{crn}</h3>"))
    if instructor_name:
        display(HTML(f"<div style='font-size: 0.95em; color: #555; margin-bottom: 10px;'>Instructor: {instructor_name}</div>"))

    instructions_html = """
    <div style="border-left: 4px solid #4CAF50; padding: 10px 15px; background-color: #f9f9f9; margin-bottom: 15px; font-size: 0.9em;">
      <p><strong>Quick Tips:</strong></p>
      <ul style="margin-top: 5px; padding-left: 20px;">
        <li>Use the score buttons to assign SLO scores for each student.</li>
        <li>Required SLOs are marked with a ⭐ symbol.</li>
        <li>You can use the toggles below to hide optional outcomes or show/hide full descriptions.</li>
      </ul>
      <p>Need more help? <a href="https://example.com/tutorial" target="_blank">Watch the full tutorial video</a>.</p>
    </div>
    """
    display(HTML(instructions_html))

    required_only_toggle = widgets.Checkbox(value=True, description="Show only SLOs required for Summer 2025", indent=False)
    description_toggle = widgets.Checkbox(value=True, description="Show full SLO descriptions", indent=False)
    display(widgets.HBox([required_only_toggle, description_toggle], layout=widgets.Layout(gap="30px", margin="10px 0")))

    students_per_page_dropdown = widgets.Dropdown(
        options=[("All", "all"), ("1", 1), ("5", 5), ("10", 10), ("20", 20), ("Custom", "custom")],
        value=5,
        description="Students per page:",
        layout=widgets.Layout(width="300px"),
        style={"description_width": "auto"}
    )

    custom_input = widgets.BoundedIntText(
        value=5, min=1, max=len(students), step=1, description="Custom:",
        layout=widgets.Layout(width="200px"), visible=False
    )

    def handle_per_page_change(change):
        custom_input.layout.visibility = 'visible' if change.new == "custom" else 'hidden'
    students_per_page_dropdown.observe(handle_per_page_change, names='value')

    display(widgets.HBox([students_per_page_dropdown, custom_input], layout=widgets.Layout(gap="20px", margin="10px 0")))

    current_page = widgets.IntText(value=1, description="Page:", layout=widgets.Layout(width="130px"))

    def get_per_page():
        val = students_per_page_dropdown.value
        if val == "custom": return custom_input.value
        elif val == "all": return len(students)
        return val

    def get_page_count():
        return max(1, -(-len(students) // get_per_page()))

    prev_button = widgets.Button(description="◀ Prev")
    next_button = widgets.Button(description="Next ▶")
    prev_button.on_click(lambda b: current_page.set_trait('value', max(1, current_page.value - 1)))
    next_button.on_click(lambda b: current_page.set_trait('value', min(get_page_count(), current_page.value + 1)))

    display(widgets.HBox([
        widgets.HBox([prev_button, next_button, current_page])
    ], layout=widgets.Layout(justify_content="flex-end", margin="0 0 10px")))

    per_page = get_per_page()
    start = (current_page.value - 1) * per_page
    end = start + per_page
    paged_students = students[start:end]

    display(HTML(f"""
      <div style='margin: 5px 0 15px; font-size: 0.85em; color: #666;'>
        Showing {len(paged_students)} of {len(students)} students
        (Page {current_page.value} of {get_page_count()})
      </div>
    """))

    for student in paged_students:
        sid = student["id"]
        student_scores[sid] = {}
        display(HTML(f"<b>{student['name']}</b>"))

        all_oids = [(oid, "required") for oid in outcomes["required"]]
        if not required_only_toggle.value:
            all_oids += [(oid, "optional") for oid in outcomes["optional"]]

        def get_slo_sort_key(oid):
            title = slo_metadata.get(str(oid), {}).get("title", "")
            _, slo_num = parse_course_and_slo_number(title)
            try:
                return int(slo_num)
            except:
                return 999

        all_oids.sort(key=lambda pair: get_slo_sort_key(pair[0]))

        for idx, (oid, req_type) in enumerate(all_oids, start=1):
            outcome_info = slo_metadata.get(str(oid), {})
            title = outcome_info.get("title", f"SLO {idx}")
            _, slo_num = parse_course_and_slo_number(title)
            description_text = outcome_descriptions.get(str(oid), "No description found.")
            is_required = (req_type == "required")
            label_text = f"⭐ SLO {slo_num}:" if is_required else f"SLO {slo_num}:"

            slo_label = widgets.HTML(value=f"<div style='font-weight: bold; width: 90px'>{label_text}</div>")
            desc_html = description_text if description_toggle.value else ""
            slo_desc = widgets.HTML(value=f"<div style='font-size: 0.9em; color: #444; line-height: 1.4;'>{desc_html}</div>")

            score_toggle = widgets.ToggleButtons(
                options=[(label, value) for label, value in score_options],
                value=None,
                button_style='',
                layout=widgets.Layout(margin="0 0 10px 0")
            )

            student_scores[sid][oid] = score_toggle

            row = widgets.HBox([
                slo_label,
                widgets.Box([slo_desc], layout=widgets.Layout(flex="1")),
                score_toggle
            ], layout=widgets.Layout(margin="6px 0", align_items="flex-start"))
            display(row)

    display(HTML("<hr style='margin: 25px 0 10px;'>"))
    display(widgets.HBox([
        widgets.HBox([prev_button, next_button, current_page])
    ], layout=widgets.Layout(justify_content="flex-end", margin="0 0 15px")))

    csv_toggle = widgets.Checkbox(value=True, description="Export CSV of Scores")
    output_area = widgets.Output()

    def on_submit_click(b):
        missing_entries = []
        for student in students:
            sid = student["id"]
            name = student["name"]
            for oid in outcomes["required"]:
                if student_scores[sid][oid].value is None:
                    _, slo_num = parse_course_and_slo_number(slo_metadata.get(str(oid), {}).get("title", ""))
                    missing_entries.append(f"{name} — SLO {slo_num}")

        with output_area:
            output_area.clear_output()
            if missing_entries:
                warning_html = "<p style='color: red; font-weight: bold;'>⚠️ Cannot submit. Missing scores for required outcomes:</p><ul>"
                for entry in missing_entries:
                    warning_html += f"<li>{entry}</li>"
                warning_html += "</ul>"
                confirm_button = widgets.Button(description="Submit Anyway", button_style='warning')

                def force_submit(_):
                    confirm_button.layout.display = 'none'
                    submit_scores()

                confirm_button.on_click(force_submit)
                display(HTML(warning_html))
                display(confirm_button)
            else:
                submit_scores()

    def submit_scores():
        with output_area:
            output_area.clear_output()
            if csv_toggle.value:
                csv_buffer = io.StringIO()
                writer = csv.writer(csv_buffer)
                writer.writerow(["Student Name", "Student ID", "SLO #", "Outcome ID", "Score", "Required/Optional", "Description"])

                for student in students:
                    sid = student["id"]
                    name = student["name"]
                    for oid, score_widget in student_scores[sid].items():
                        outcome_info = slo_metadata.get(str(oid), {})
                        _, slo_num = parse_course_and_slo_number(outcome_info.get("title", ""))
                        desc = outcome_descriptions.get(str(oid), "")
                        score = score_widget.value
                        req_status = "Required" if oid in outcomes["required"] else "Optional"
                        writer.writerow([name, sid, slo_num, oid, score, req_status, desc])

                filename = f"SLO_Scores_{matching_section.get('sis_section_id', 'UNKNOWN')}.csv"
                with open(filename, "w", newline='') as f:
                    f.write(csv_buffer.getvalue())
                display(HTML("<p>✅ CSV download ready:</p>"))
                display(FileLink(filename))
            else:
                display(HTML("<p>✅ Scores submitted successfully (no CSV exported).</p>"))

    submit_button = widgets.Button(description="Submit Scores", button_style='success')
    submit_button.on_click(on_submit_click)

    display(widgets.VBox([
        widgets.HBox([submit_button, csv_toggle], layout=widgets.Layout(justify_content="center", gap="20px")),
        output_area
    ], layout=widgets.Layout(margin="20px 0")))


In [10]:
slo_metadata = {
    "1400300": {
        "title": "ART 001A SLO 1. Research & Information Literacy, Quantitative Reasoning",
        "description": "Describe and analyze works of art and architecture, employing the language of formal and contextual analysis modeled in class and in assigned reading."
    },
    "1400301": {
        "title": "ART 001A SLO 2. Research & Information Literacy",
        "description": "Identify the key stylistic and thematic features of a work of art or architecture from a given period, cultural tradition and regional location of prehistory through the middle ages, including unknown works."
    },
    "1400302": {
        "title": "ART 001A SLO 3. Research & Information Literacy",
        "description": "Evaluate works of art and/or architecture, accounting for both similarities and differences."
    },
    "1400303": {
        "title": "ART 001A SLO 4. Quantitative Reasoning",
        "description": "Analyze the ways in which art from prehistory through the middle ages has been employed historically to express fundamental human ideals, values, and beliefs."
    }
}


In [11]:
instructor_name = get_instructor_name()  # 🆕 Add this line!

# Check teacher enrollment before rendering
if is_user_teacher(COURSE_ID):
    render_slo_interface(
        parsed_course_code,
        section_id,
        students,
        outcomes,
        slo_metadata,
        outcome_descriptions,
        sections,
        instructor_name
    )
else:
    from IPython.display import display, HTML
    display(HTML("<div style='color: red; font-weight: bold; margin: 10px 0;'>⛔ You are not enrolled as a Teacher in this course.</div>"))
